In [1]:
import os
import pandas as pd
import geopandas as gpd

local_crs = 32636
example_data_path = "./data"

In [2]:
# load aggregated info we have
aggregated_blocks = gpd.read_parquet(os.path.join(example_data_path, "data_getter_blocks.parquet"))
accessibility_matrix = pd.read_pickle(os.path.join(example_data_path, "data_getter_matrix.pickle"))

In [3]:
from masterplan_tools.models import City

city_model = City(
  blocks_gdf=aggregated_blocks, 
  matrix=accessibility_matrix
)

In [ ]:
city_model.to_pickle(os.path.join(example_data_path, 'city_model.pickle'))

In [4]:
schools = gpd.read_parquet(os.path.join(example_data_path, "schools.parquet"))
kindergartens = gpd.read_parquet(os.path.join(example_data_path, "kindergartens.parquet"))
city_model.update_layer('schools', schools)
city_model.update_layer('kindergartens', kindergartens)

In [5]:
city_model[16]

Block(id=16, geometry=<POLYGON ((344272.022 6648285.316, 344278.598 6648288.103, 344286.59 6648291...>, population=3946, floors=3.5, area=102276.4154130563, living_area=19338.336, green_area=1900.0, industrial_area=7108.819499999998, green_capacity=4095, parking_capacity=44, capacities={ServiceType(name='schools', accessibility=15, demand=120, buffer=0): 1200}, city=<masterplan_tools.models.city.City object at 0x7f0764dbbac0>)